In [1]:
with open('SP_pair.txt', 'r') as f:
    ppi_pairs = f.readlines() 


In [2]:
# !mkdir CV

In [2]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import numpy as np
for rep in range(3):
    n_splits = 10
    y_all_1 =  np.zeros((int(len(ppi_pairs)/2),1))
    y_all_2 =  np.ones( ( len(ppi_pairs) - int(len(ppi_pairs)/2)   ,1))
    y_all = np.concatenate((y_all_1, y_all_2), axis=0)
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True)
    skf_v = StratifiedKFold(n_splits=5, shuffle=True)

    for i, (train, test) in enumerate(skf.split(ppi_pairs, y_all)):
         
        train_pairs_file = 'CV/train'+str(rep)+'-'+str(i)
        test_pairs_file = 'CV/test'+str(rep)+'-'+str(i)
        valid_pairs_file = 'CV/valid'+str(rep)+'-'+str(i)
        valid_len = int(train.shape[0]*0.1)
        train_valid_idx = np.arange(train.shape[0])
        np.random.shuffle(train_valid_idx)
         
         
        with open(train_pairs_file,'w') as f:
            for ind in train_valid_idx[:train.shape[0]-valid_len]:
                f.writelines(ppi_pairs[ind])
         
        with open(test_pairs_file,'w') as f:
            for ind in test:
                f.writelines(ppi_pairs[ind])
        
        with open(valid_pairs_file,'w') as f:
            for ind in train_valid_idx[train.shape[0]-valid_len:]:
                f.writelines(ppi_pairs[ind])

In [4]:
with open('sequence.txt', 'r') as f:
    all_seqs = f.readlines()

In [5]:
all_seqs[0]

'Q09682\tMSRSYDSRTTIFSPEGRLYQVEYALEAINHAGVALGIVAKDGIVLAAEKKVTSKLLEQEESAEKLYHIGDNMLCAVAGLTADANILINYARRVGQQYLQTFNEEMPCEQLVRRVCDLKQGYTQYGGLRPFGVSFLYAGWDHIRGYQLFQSNPSGNYGSWQANSIGGNSTSVQSLMRQEYKDDINLDEASAMAVKFLSKTLDSNSLTHEKIEFATITKDTTKNKMVCKIWKSDEINEVLNKYQETQRQS\n'

In [6]:
prot2seq = {}
for line in all_seqs:
    protID, seqs = line.rstrip().split('\t')
    prot2seq[protID] = seqs

In [7]:
prot2seq['Q09682']

'MSRSYDSRTTIFSPEGRLYQVEYALEAINHAGVALGIVAKDGIVLAAEKKVTSKLLEQEESAEKLYHIGDNMLCAVAGLTADANILINYARRVGQQYLQTFNEEMPCEQLVRRVCDLKQGYTQYGGLRPFGVSFLYAGWDHIRGYQLFQSNPSGNYGSWQANSIGGNSTSVQSLMRQEYKDDINLDEASAMAVKFLSKTLDSNSLTHEKIEFATITKDTTKNKMVCKIWKSDEINEVLNKYQETQRQS'

In [8]:
from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di

In [9]:
save_dict(prot2seq, 'SPprot2seq.pkl')

In [10]:
with open('go_term.txt', 'r') as f:
    all_goterms = f.readlines()

In [11]:
all_goterms[0]

'P17871\tGO:0000983,GO:0005669,GO:0000120,GO:0006367,GO:0006384,GO:0006355,GO:0000126,GO:1903357,GO:0005634,GO:0001169,GO:0003677,GO:0006360,GO:0001006,GO:0000979\n'

In [12]:
prot2go= {}
for line in all_goterms:
    splittext = line.rstrip().split('\t')
    if len(splittext) == 2:
        protID, gos = line.rstrip().split('\t')
        prot2go[protID] = gos
    else:
        print(line)
        protID = splittext[0] 
        prot2go[protID] = ''

In [13]:
prot2go['P17871']

'GO:0000983,GO:0005669,GO:0000120,GO:0006367,GO:0006384,GO:0006355,GO:0000126,GO:1903357,GO:0005634,GO:0001169,GO:0003677,GO:0006360,GO:0001006,GO:0000979'

In [14]:
save_dict(prot2go, 'SPprot2go.pkl')

In [20]:
max_golen = 0
for key, value in prot2go.items():
    gos = value.split(',')
    count = 0
    for go in gos:
        feature = np.load('../ncbi_allfeatures4go/'+go+'_0.npy')
        count += feature.shape[0]-2
    if max_golen < count:
        max_golen = count
    

In [21]:
max_golen

417